## load packages

In [17]:
from sklearn.metrics import confusion_matrix
import load_data
import my_metrics
import plot_learning_curve
import numpy as np
import sys 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn import ensemble
from sklearn.metrics import mean_squared_error
from xgboost import XGBClassifier
import pandas as pd
from sklearn import metrics
from sklearn import tree
import matplotlib.pyplot as plt 
from sklearn.metrics import f1_score, precision_score, recall_score
import seaborn as sns
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import matplotlib as mpl

%matplotlib inline

## load goamazon

In [18]:
dilute_cape = np.loadtxt("../../data/goamazon/dilute_cape")
dilute_dcape = np.loadtxt("../../data/goamazon/dilute_dcape")
undilute_dcape = np.loadtxt("../../data/goamazon/undilute_dcape")
undilute_cape = np.loadtxt("../../data/goamazon/undilute_cape")
lcl = np.loadtxt("../../data/goamazon/dilute_lcl")

dataset_goamazon = load_data.load_arm_hy("../../data/goamazon/trigger_goamazon_hy.nc")

dataset_goamazon['cape'] = dilute_dcape
dataset_goamazon['lcl'] = lcl
dataset_goamazon['where'] = 0.0
print(dataset_goamazon.shape)
#print(dataset.dtypes)

dataset_goamazon.rename(columns={'cape': 'ddcape'}, inplace=True)

pos_goamazon = dataset_goamazon[dataset_goamazon.label==1]
neg_goamazon = dataset_goamazon[dataset_goamazon.label==0]
print(pos.shape)
print(neg.shape)

trig_x_goamazon = dataset_goamazon.iloc[:,0:86]
trig_y_goamazon = dataset_goamazon.iloc[:,86]

print(np.sum(trig_y_goamazon))
trig_x_train_goamazon,trig_x_test_goamazon,trig_y_train_goamazon,trig_y_test_goamazon= train_test_split(trig_x_goamazon, trig_y_goamazon, test_size=0.2, random_state=20)

(5840, 93)
(935, 92)
(4905, 92)
935.0


## load SGP

In [19]:
dilute_cape = np.loadtxt("../../data/sgp/arm_continuous_at_sgp/dilute_cape")
dilute_dcape = np.loadtxt("../../data/sgp/arm_continuous_at_sgp/dilute_dcape")
undilute_dcape = np.loadtxt("../../data/sgp/arm_continuous_at_sgp/undilute_dcape")
undilute_cape = np.loadtxt("../../data/sgp/arm_continuous_at_sgp/undilute_cape")
lcl = np.loadtxt("../../data/sgp/arm_continuous_at_sgp/dilute_lcl")

dataset_sgp = load_data.load_sgp_data("../../data/sgp/arm_continuous_at_sgp/Arm_CF_1999_2009_hy.nc")
dataset_sgp['cape'] = dilute_dcape
dataset_sgp['lcl'] = lcl
dataset_sgp['where'] = 1.0
dataset_sgp.rename(columns={'cape': 'ddcape'}, inplace=True)
print(dataset_sgp.shape)

trig_x_sgp = dataset_sgp.iloc[:,0:86]
trig_y_sgp = dataset_sgp.iloc[:,86]

pos_sgp = dataset_sgp[dataset_sgp.label==1]
neg_sgp = dataset_sgp[dataset_sgp.label==0]

print(trig_x_sgp.shape)
print(pos_sgp.shape)
print(neg_sgp.shape)

trig_x_train_sgp,trig_x_test_sgp,trig_y_train_sgp,trig_y_test_sgp= train_test_split(trig_x_sgp, trig_y_sgp, test_size=0.2, random_state=20)

(22800, 93)
(22800, 86)
(1991, 93)
(20809, 93)


## SGP train, GoAmazon test

In [11]:
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=10)
xgb = XGBClassifier(n_estimators=600,silent=True, nthread=8, max_depth=7,scale_pos_weight=3.5)

xgb.fit(trig_x_sgp,trig_y_sgp)
y_pred_amazon = xgb.predict(trig_x_test_goamazon)
tn, fp, fn, tp = confusion_matrix(trig_y_test_goamazon, y_pred_amazon).ravel()
f1 = f1_score(trig_y_test_goamazon, y_pred_amazon,average='macro')
P  = precision_score(trig_y_test_goamazon, y_pred_amazon,average='macro')
R  = recall_score(trig_y_test_goamazon, y_pred_amazon,average='macro')
print(tn,fp,fn,tp)
print(f1, P, R)

398 573 72 125
0.41586188982666444 0.5129458025970859 0.5222022406122737


## GoAmazon train, SGP test

In [12]:
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=10)
xgb = XGBClassifier(n_estimators=600,silent=True, nthread=8, max_depth=7,scale_pos_weight=3.5)

xgb.fit(trig_x_goamazon,trig_y_goamazon)
y_pred_sgp = xgb.predict(trig_x_test_sgp)
tn, fp, fn, tp = confusion_matrix(trig_y_test_sgp, y_pred_sgp).ravel()
f1 = f1_score(trig_y_test_sgp, y_pred_sgp,average='macro')
P  = precision_score(trig_y_test_sgp, y_pred_sgp,average='macro')
R  = recall_score(trig_y_test_sgp, y_pred_sgp,average='macro')
print(tn,fp,fn,tp)
print(f1, P, R)

3918 220 166 256
0.7616043436074327 0.7485843504884814 0.7767346452598618


## Joint train, Joint test

In [30]:
joint = pd.concat([dataset_goamazon,dataset_sgp], sort=False)

print(joint.shape)

x_joint = joint.iloc[:,0:86]
y_joint = joint.iloc[:,86]
x_train_joint,x_test_joint,y_train_joint,y_test_joint= train_test_split(x_joint, y_joint, test_size=0.2, random_state=20)


cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=10)
xgb = XGBClassifier(n_estimators=600,silent=True, nthread=8, max_depth=7,scale_pos_weight=3.5)

xgb.fit(x_train_joint,y_train_joint)
y_pred_joint = xgb.predict(x_test_joint)
tn, fp, fn, tp = confusion_matrix(y_test_joint, y_pred_joint).ravel()
f1 = f1_score(y_test_joint, y_pred_joint,average='macro')
P  = precision_score(y_test_joint, y_pred_joint,average='macro')
R  = recall_score(y_test_joint, y_pred_joint,average='macro')
print(tp,fp,fn,tn)
print(f1, P, R)

(28640, 93)
415 76 134 5103
0.8889578163771712 0.9098133405305426 0.8706226033325701


## Joint train,  SGP test

In [33]:
x_joint_train = np.concatenate((trig_x_train_sgp, trig_x_train_goamazon), axis=0)
y_joint_train = np.concatenate((trig_y_train_sgp, trig_y_train_goamazon), axis=0)

cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=10)
xgb = XGBClassifier(n_estimators=600,silent=True, nthread=8, max_depth=7,scale_pos_weight=3.5)

xgb.fit(x_joint_train, y_joint_train)
y_pred_sgp = xgb.predict(trig_x_test_sgp.values)
tn, fp, fn, tp = confusion_matrix(trig_y_test_sgp, y_pred_sgp).ravel()
f1 = f1_score(trig_y_test_sgp, y_pred_sgp,average='macro')
P = precision_score(trig_y_test_sgp, y_pred_sgp,average='macro')
R = recall_score(trig_y_test_sgp, y_pred_sgp,average='macro')
print(tp,fp,fn,tn)
print(f1, P, R)

339 47 83 4091
0.9117344361962664 0.9291766697865642 0.8959796957570454


## Joint train, Goamazon test

In [34]:
x_joint_train = np.concatenate((trig_x_train_sgp, trig_x_train_goamazon), axis=0)
y_joint_train = np.concatenate((trig_y_train_sgp, trig_y_train_goamazon), axis=0)

cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=10)
xgb = XGBClassifier(n_estimators=600,silent=True, nthread=8, max_depth=7,scale_pos_weight=3.5)

xgb.fit(x_joint_train, y_joint_train)
y_pred_goamazon = xgb.predict(trig_x_test_goamazon.values)
tn, fp, fn, tp = confusion_matrix(trig_y_test_goamazon, y_pred_goamazon).ravel()
f1 = f1_score(trig_y_test_goamazon, y_pred_goamazon,average='macro')
P = precision_score(trig_y_test_goamazon, y_pred_goamazon,average='macro')
R = recall_score(trig_y_test_goamazon, y_pred_goamazon,average='macro')
print(tp,fp,fn,tn)
print(f1, P, R)

133 17 64 954
0.8629233116434337 0.9118991486574983 0.8288095897787096
